In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
# llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [30]:
prompt=ChatPromptTemplate.from_template(
                        template="Tell me a joke about {topic}")

In [5]:
chain=prompt|llm|StrOutputParser()
chain.invoke(input={'topic':"Bears"})

"Why don't bears wear shoes?\n\nBecause they already have bear feet!"

In [32]:
chainRitzStyle=RunnablePassthrough.assign(topic=itemgetter("question")|prompt).assign(result=itemgetter('topic')|llm).assign(output=itemgetter('result')|StrOutputParser())|RunnableLambda(lambda x:x['output'])

In [33]:
chainRitzStyle.invoke(input={'question':"Bears"})

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the drama!"

In [18]:
crs=RunnablePassthrough.assign(topic=itemgetter('question')|prompt).assign(result=itemgetter("topic")|llm).assign(output=itemgetter('result')|StrOutputParser()).assign(runnable_response=itemgetter('output')|RunnableLambda(func=lambda x: x.upper()))

In [19]:
crs.invoke(input={'question':'Bears'})

{'question': 'Bears',
 'topic': ChatPromptValue(messages=[HumanMessage(content='Tell me a joke about Bears', additional_kwargs={}, response_metadata={})]),
 'result': AIMessage(content='Why do bears have hairy coats?\n\nFur protection!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cxes4lAcVPhGhiSLdBY5TsJ0sPaoZ', 'finish_reason': 'stop', 'logprobs': None}, id='run--569e1664-1cd0-414c-8125-b56a565fd704-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'output': 'Why do bears 

<h3> Coercion</h3>

In [20]:
analysisPrompt=ChatPromptTemplate.from_template(
    template="Is this a funny joke? {joke}"
)

In [21]:
combinedChain={"joke":chainRitzStyle}|analysisPrompt|llm|StrOutputParser()

In [22]:
combinedChain.invoke(input={'question':"Bears"})

'Yes, this joke is funny because it plays on the idea of fast food being quick and difficult to catch for animals like bears who are used to hunting for their food.'

In [24]:
combinedChain_crs={"joke":crs}|analysisPrompt|llm|StrOutputParser()
combinedChain_crs.invoke(input={'question':"Bears"})

'Some people might find this joke about a bear wanting to be a hot dog funny, while others may not. Humor is subjective, so it ultimately depends on your sense of humor!'

<h3> Using the Pipe Method</h3>

In [25]:
composedChainWithPipe=RunnablePassthrough.assign(joke=chainRitzStyle).pipe(analysisPrompt).pipe(llm).pipe(StrOutputParser())

In [26]:
composedChainWithPipe.invoke(input={'question':"Bears"})

'Some people might find this joke funny, as it plays on the words "bear" (referring to the animal) and "bare" (meaning uncovered or exposed). This plays on a pun, which can be humorous to some.'